In [1]:
!pip install pygame==2.0.0.dev10
!pip install qiskit
!pip install quantumrandom

In [2]:
import random
import pygame
import math
import sys
import copy
from pygame.locals import *
from pygame.math import Vector2
from math import radians, sin, cos

from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, execute, Aer
import quantumrandom as qr

pygame 2.0.0.dev10 (SDL 2.0.12, python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
FPS = 30
WINDOWWIDTH = 640
WINDOWHEIGHT = 480

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
DARKGREEN = (0, 155, 0)
DARKGRAY = (40, 40, 40)
BGCOLOR = BLACK

UP = 'up'
DOWN = 'down'
LEFT = 'left'
RIGHT = 'right'

UPLEFT = 'upleft'
UPRIGHT = 'upright'
DOWNLEFT = 'downleft'
DOWNRIGHT = 'downright'


#difficulty: speed [0],  protons/neutrons [1], rotation speed [2], max electrons [3], na [4], enemy speed [5]
EASY = (0.07, 1, 0, 15, 0, 1)
MEDIUM = (0.1, 3, 0, 10, 0, 3) 
HARD = (0.15, 5, 0, 8, 0, 5)

        
#levels = ( (atom name, protons, electrons, fun fact) )
levels = (
        ("HYDROGEN", 1, 1, "Most common element in universe"), 
        ("HELIUM", 2, 2, "Rare on earth"),
        ("LITHIUM", 3, 3, "It burns white (don't do this)"),
        ("BERYLLIUM", 4, 4, "Found in gemstones (like emerald)"),
        ("BORON", 5, 5, "Identified on surface of Moon & Mars"),
        ("CARBON", 6, 6, "Different forms - like diamonds (hardest element)"),
        ("NITROGEN", 7, 7, "78.1% of Earth's air"),
        ("OXYGEN", 8, 8, "Most numerous atom in human body"),
        ("FLUORINE", 9, 9, "Can attack diamond"),
        ("NEON", 10, 10, "10 litres in a new home"),
        ("SODIUM", 11, 11, "Explodes in water (don't try this)"),
        ("MAGNESIUM", 12, 12, "Found in human skeleton"),
        ("ALUMINUM", 13, 13, "Used to be more precious than gold"),
        ("SILICON", 14, 14, "Used to produce steel"),
        ("PHOSPHORUS", 15, 15, "Used in fertilizer"),
        ("SULFUR", 16, 16, "Some organisms use it as energy source"),
        ("CHLORINE", 17, 17, "Makes drinking water safe"),
        ("ARGON", 18, 18, "Glows blue-violet when excited")
)

In [4]:
def main():
    global FPSCLOCK, DISPLAYSURF, BASICFONT, heart, diff
    
    pygame.init()
    FPSCLOCK = pygame.time.Clock()
    DISPLAYSURF = pygame.display.set_mode((WINDOWWIDTH, WINDOWHEIGHT))
    BASICFONT = pygame.font.Font('freesansbold.ttf', 18)
    pygame.display.set_caption('Electroquantum')
    
    heart = 5
    diff = MEDIUM
    
    showStartScreen()
    showInstructions()
    
    while True:
        selectLevel()
        runGame()

In [5]:
def showStartScreen():
    
    global titleFont
    
    titleFont = pygame.font.Font('freesansbold.ttf', 100)
    electroSurf = titleFont.render('ELECTRO', True, (50, 128, 246))
    electroRect = electroSurf.get_rect()
    electroRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2-50)
    quantumSurf = titleFont.render('QUANTUM', True, (50, 128, 246))
    quantumRect = quantumSurf.get_rect()
    quantumRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2+50)
    startSurf = BASICFONT.render('Start', True, (50, 128, 246), (255, 255, 255))
    startRect = startSurf.get_rect()
    startRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2+200)
    
    DISPLAYSURF.fill((248, 185, 129))
  
    DISPLAYSURF.blit(electroSurf, electroRect)
    DISPLAYSURF.blit(quantumSurf, quantumRect)
    DISPLAYSURF.blit(startSurf,startRect)
        
    pygame.display.update()
    FPSCLOCK.tick(FPS)
    
    while True:
    
        for event in pygame.event.get():
            if event.type == MOUSEBUTTONUP:
                mousex, mousey = event.pos
                if pygame.Rect(startRect).collidepoint(mousex, mousey):
                    return #start game
    

In [6]:
def showInstructions():
    instructions = "The goal of the game is to collect \n the number of electrons in the atom.\n  Make it through each level to win.\n  Electrons don't follow the laws of classical mechanics \n because they are so small, so they may \n move to a random position when you approach. \n Avoid protons or neutrons. \n Protons are blue, neutrons are green, and electrons are white."
    
    startSurf = BASICFONT.render('Continue', True, (50, 128, 246), (255, 255, 255))
    startRect = startSurf.get_rect()
    startRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2+200)
    
    DISPLAYSURF.fill((248, 185, 129))
    DISPLAYSURF.blit(startSurf,startRect)
    
    lines = instructions.splitlines()
    for i, l in enumerate(lines):
        surf = BASICFONT.render(l, True, (50, 128, 246))
        rect = surf.get_rect()
        rect.center = (WINDOWWIDTH/2, 100 + i * 30)
        DISPLAYSURF.blit(surf, rect)
    
    pygame.display.update()
    FPSCLOCK.tick(FPS)
    
    while True:
    
        for event in pygame.event.get():
            if event.type == MOUSEBUTTONUP:
                mousex, mousey = event.pos
                if pygame.Rect(startRect).collidepoint(mousex, mousey):
                    return #continue game

In [7]:
def selectLevel():
    levelSurf = BASICFONT.render('Choose a level', True, (50, 128, 246))
    levelRect = levelSurf.get_rect()
    levelRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2-200)
    
    easySurf = BASICFONT.render('Easy', True, (50, 128, 246), (255, 255, 255))
    easyRect = easySurf.get_rect()
    mediumSurf = BASICFONT.render('Medium', True, (50, 128, 246), (255, 255, 255))
    mediumRect = mediumSurf.get_rect()
    hardSurf = BASICFONT.render('Hard', True, (50, 128, 246), (255, 255, 255))
    hardRect = hardSurf.get_rect()
    
    easyRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2)
    mediumRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2+50)
    hardRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2+100)
    
    DISPLAYSURF.fill((248, 185, 129))
    
    DISPLAYSURF.blit(levelSurf, levelRect)
    DISPLAYSURF.blit(easySurf, easyRect)
    DISPLAYSURF.blit(mediumSurf, mediumRect)
    DISPLAYSURF.blit(hardSurf, hardRect)
    
    pygame.display.update()
    FPSCLOCK.tick(FPS)
    
    while True:
    
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == MOUSEBUTTONUP:
                    mousex, mousey = event.pos
                    if pygame.Rect(easyRect).collidepoint(mousex, mousey):
                        diff = EASY
                        return # next
                    elif pygame.Rect(mediumRect).collidepoint(mousex, mousey):
                        diff = MEDIUM
                        return # next
                    elif pygame.Rect(hardRect).collidepoint(mousex, mousey):
                        diff = HARD
                        return # next

In [8]:
def runGame():
    

    for level in levels:
        winner = playGame(level)
        if winner == False:
            break
            
    ## add in win screen here
    
    if winner:
        
        titleFont = pygame.font.Font('freesansbold.ttf', 100)
        electroSurf = titleFont.render('WINNER', True, (50, 128, 246))
        electroRect = electroSurf.get_rect()
        electroRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2-50)
        quantumSurf = titleFont.render('WINNER', True, (50, 128, 246))
        quantumRect = quantumSurf.get_rect()
        quantumRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2+50)
        startSurf = BASICFONT.render('Replay?', True, (50, 128, 246), (255, 255, 255))
        startRect = startSurf.get_rect()
        startRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2+200)

        DISPLAYSURF.fill((248, 185, 129))

        DISPLAYSURF.blit(electroSurf, electroRect)
        DISPLAYSURF.blit(quantumSurf, quantumRect)
        DISPLAYSURF.blit(startSurf,startRect)

        pygame.display.update()
        FPSCLOCK.tick(FPS)
        
    else:
        titleFont = pygame.font.Font('freesansbold.ttf', 100)
        electroSurf = titleFont.render('SORRY', True, (50, 128, 246))
        electroRect = electroSurf.get_rect()
        electroRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2-50)
        quantumSurf = titleFont.render('TRY AGAIN', True, (50, 128, 246))
        quantumRect = quantumSurf.get_rect()
        quantumRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2+50)
        startSurf = BASICFONT.render('Play?', True, (50, 128, 246), (255, 255, 255))
        startRect = startSurf.get_rect()
        startRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2+200)

        DISPLAYSURF.fill((248, 185, 129))

        DISPLAYSURF.blit(electroSurf, electroRect)
        DISPLAYSURF.blit(quantumSurf, quantumRect)
        DISPLAYSURF.blit(startSurf,startRect)

        pygame.display.update()
        FPSCLOCK.tick(FPS)
    
    while True:
    
        for event in pygame.event.get():
            if event.type == MOUSEBUTTONUP:
                mousex, mousey = event.pos
                if pygame.Rect(startRect).collidepoint(mousex, mousey):
                    return #restart game

In [9]:
def playGame(level):
    displayAtom(level)
    win = electronGo(level)
    
    if win:
        return True
    else:
        return False

In [10]:
def displayAtom(level):
    atomFont = pygame.font.Font('freesansbold.ttf', 100)
    atomSurf = titleFont.render(level[0], True, (50, 128, 246))
    atomRect = atomSurf.get_rect()
    atomRect.center = (WINDOWWIDTH/2, 100)
    protonSurf = BASICFONT.render("Protons: " + str(level[1]), True, (50, 128, 246))
    protonRect = protonSurf.get_rect()
    protonRect.center = (WINDOWWIDTH/2, 200)
    electroSurf = BASICFONT.render('Electrons: ' + str(level[2]), True, (50, 128, 246))
    electroRect = electroSurf.get_rect()
    electroRect.center = (WINDOWWIDTH / 2, 250)
    factSurf = BASICFONT.render("Fun fact: " + str(level[3]), True, (50, 128, 246))
    factRect = factSurf.get_rect()
    factRect.center = (WINDOWWIDTH/2, 300)
    startSurf = BASICFONT.render('Start level', True, (50, 128, 246), (255, 255, 255))
    startRect = startSurf.get_rect()
    startRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2+200)
    
    DISPLAYSURF.fill((248, 185, 129))
    
    DISPLAYSURF.blit(atomSurf, atomRect)
    DISPLAYSURF.blit(protonSurf, protonRect)
    DISPLAYSURF.blit(electroSurf, electroRect)
    DISPLAYSURF.blit(factSurf, factRect)
    DISPLAYSURF.blit(startSurf,startRect)

    pygame.display.update()
    FPSCLOCK.tick(FPS)
    
    while True:
    
        for event in pygame.event.get():
            if event.type == MOUSEBUTTONUP:
                mousex, mousey = event.pos
                if pygame.Rect(startRect).collidepoint(mousex, mousey):
                    return #start level

In [11]:
def electronGo(level):
    
    global heart, player, playerRect, playerCoords, angle, direc, center, original
    
    device = 'qasm_simulator'
        
    simulator = Aer.get_backend(device)
    
    c = ClassicalRegister(1)
    q = QuantumRegister(1)
    circ = QuantumCircuit(q, c)
    circ.measure(q, c)
    circ.h(q[0])
    circ.measure(q, c)
    
    loadingScreen()
    
    ## 
    
    electrons = getElectrons()
    protons = getProtons()
    neutrons = getNeutrons()
    
    direction = None
    
    electronNum = level[2]
    
    gathered = 0
    
    player = pygame.image.load('spaceship.png').convert()
    original = player
    playerRect = player.get_rect()
    
    angle = 0
    speed = 0.04
    rotation_speed = 0.08
    direc = 0
    delta = 0.0
    setDirection()
    playerRect.center = (250, 250)
    playerCoords = pygame.Vector2(playerRect.center)

        
    while True:
        
        for event in pygame.event.get():
            if event.type == QUIT:
                terminate()
            elif event.type == KEYDOWN:
                if (event.key == K_LEFT or event.key == K_a) and direction != RIGHT:
                    direction = LEFT
                elif (event.key == K_RIGHT or event.key == K_d) and direction != LEFT:
                    direction = RIGHT
                elif event.key == K_ESCAPE:
                    terminate()
            else:
                direction = UP
                    
        if playerCoords[0] <= -1 or playerCoords[0]  >= WINDOWWIDTH or playerCoords[1]<= -1 or playerCoords[1]>=WINDOWHEIGHT:
            heart = heart - 1
            if heart <= 0:
                return False
            playerCoords = pygame.Vector2((250, 250))

                            
        for i, electron in enumerate(electrons):
            if pygame.Rect(playerRect).collidepoint(electron['x'], electron['y']):
                gathered = gathered + 1
                if gathered == electronNum:
                    return True
                x = random.randint(0, WINDOWWIDTH)
                y = random.randint(0, WINDOWHEIGHT)
                circle_surface = {'x': x, 'y': y}
                electrons[i] = circle_surface                
            elif dist(playerCoords, electron) == 30:    
                result = execute(circ, backend=simulator, shots=1).result()
                if result.get_counts(circ).get('0') is not None:
                    x = random.randint(0, WINDOWWIDTH)
                    y = random.randint(0, WINDOWHEIGHT)
                    circle_surface = {'x': x, 'y': y}
                    electrons[i] = circle_surface   
                
        for enemy in protons+neutrons:
            if pygame.Rect(playerRect).collidepoint(enemy['x'], enemy['y']):
                heart = heart - 1
                if heart <= 0:
                    return False
                playerCoords = pygame.Vector2((250, 250))
            
        ## edit this part
        
        
        if direction == RIGHT:
            angle = (angle - rotation_speed * delta) % 360
            doRotate()
        elif direction == LEFT:
            angle = (angle + rotation_speed * delta) % 360
            doRotate()
            
        playerCoords -= direc * delta * speed  
        playerRect.center = playerCoords
                        
        protons = moveBody(protons)
        neutrons = moveBody(neutrons)
        
        DISPLAYSURF.fill(BGCOLOR)
        DISPLAYSURF.blit(player, playerRect)
        drawElectron(electrons)
        drawProton(protons)
        drawNeutron(neutrons)
        drawScore(gathered)
        drawHearts(heart)
        pygame.display.update()
        delta = FPSCLOCK.tick(FPS)
        
        if delta >= 100:
            delta = 33

In [12]:
def setDirection():
    
    global direc, angle
    
    rad = radians(angle)
    direc = pygame.Vector2(sin(rad), cos(rad))

In [13]:
def doRotate():
    
    global player, playerRect, angle, direc, playerCoords, original
    
    player = pygame.transform.rotate(original, angle)
    playerRect = player.get_rect()
    playerRect.center = playerCoords
    setDirection()

In [14]:
def loadingScreen():

    titleFont = pygame.font.Font('freesansbold.ttf', 150)
    titleSurf = titleFont.render('Loading!', True, (50, 128, 246))
    titleRect = titleSurf.get_rect()
    titleRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2)
    
    #quantumSurf = BASICFONT.render('Quantum random is slow :(', True, (50, 128, 246))
    #quantumRect = quantumSurf.get_rect()
    #quantumRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2+100)
    
    DISPLAYSURF.fill((248, 185, 129))
  
    DISPLAYSURF.blit(titleSurf, titleRect)
    #DISPLAYSURF.blit(quantumSurf,quantumRect)
        
    pygame.display.update()
    FPSCLOCK.tick(FPS)
        
    return # initialize now

In [15]:
def getElectrons():
    num = diff[3]
    electrons = []
    for i in range(num):
        x = random.randint(0, WINDOWWIDTH)
        y = random.randint(0, WINDOWHEIGHT)
        electrons.append({'x': x, 'y': y})
    return electrons

In [16]:
def getProtons():
    num = diff[1]
    directions = [UP, RIGHT, DOWN, LEFT, UPRIGHT, UPLEFT, DOWNRIGHT, DOWNLEFT]
    protons = []
    for i in range(num):
        x = random.randint(0, WINDOWWIDTH)
        y = random.randint(0, WINDOWHEIGHT)
        direct = random.randint(0, len(directions)-1)
        protons.append({'x': x, 'y': y, 'd': directions[direct]})
    return protons

In [17]:
def getNeutrons():
    num = diff[1]
    neutrons = []
    directions = [UP, RIGHT, DOWN, LEFT, UPRIGHT, UPLEFT, DOWNRIGHT, DOWNLEFT]
    for i in range(num):
        x = random.randint(0, WINDOWWIDTH)
        y = random.randint(0, WINDOWHEIGHT)
        direct = random.randint(0, len(directions)-1)
        neutrons.append({'x': x, 'y': y, 'd': directions[direct]})
    return neutrons

In [18]:
def moveBody(enemy):
    newE = []
    for i, circle_surface in enumerate(enemy):
        x = circle_surface['x']
        y = circle_surface['y']
        d = circle_surface['d']
        
        if d == UP or d == UPRIGHT or d==UPLEFT:
            if y<= -1:
                y = WINDOWHEIGHT
                x = random.randint(0, WINDOWWIDTH)
            else:
                y = y - diff[5]
        if d==RIGHT or d==UPRIGHT or d==DOWNRIGHT:
            if x  >= WINDOWWIDTH:
                x = 0
                y = random.randint(0, WINDOWHEIGHT)
            else:
                x = x + diff[5]
        if d==DOWN or d==DOWNRIGHT or d==DOWNLEFT:
            if y>=WINDOWHEIGHT:
                y = 0
                x = random.randint(0, WINDOWWIDTH)
            else:
                y = y + diff[5]
        if d==LEFT or d==UPLEFT or d==DOWNLEFT:
            if x <= -1:
                x = WINDOWWIDTH
                y = random.randint(0, WINDOWHEIGHT)
            else:
                x = x - diff[5]
            
        newE.append({'x': x, 'y': y, 'd': d})   
    return newE

In [19]:
def drawElectron(electrons):
    for e in electrons:
        pygame.draw.circle(DISPLAYSURF, WHITE, (e['x'], e['y']), 5, 0)

In [20]:
def drawProton(protons):
    for p in protons:
        pygame.draw.circle(DISPLAYSURF, BLUE, (p['x'], p['y']), 10, 0)

In [21]:
def drawNeutron(neutrons):
    for n in neutrons:
        pygame.draw.circle(DISPLAYSURF, GREEN, (n['x'], n['y']), 10, 0)

In [22]:
def drawScore(score):
    scoreSurf = BASICFONT.render('Score: %s'%(score), True, WHITE)
    scoreRect = scoreSurf.get_rect()
    scoreRect.topleft = (WINDOWWIDTH-120, 10)
    DISPLAYSURF.blit(scoreSurf, scoreRect)

In [23]:
def drawHearts(h):
    heartSurf = BASICFONT.render('Lives left: %s'%(h), True, WHITE)
    heartRect = heartSurf.get_rect()
    heartRect.topleft = (WINDOWWIDTH-120, 30)
    DISPLAYSURF.blit(heartSurf, heartRect)

In [24]:
def dist(player, electron):
    xp = player[0]
    yp = player[1]
    xe = electron['x']
    ye = electron['y']
    
    d = math.sqrt((xp-xe)**2 + (yp-ye)**2)
    return int(d)

In [25]:
def terminate():
    pygame.quit()
    sys.exit()

In [ ]:
if __name__ == '__main__':
    main()